In [41]:
import transformers
from datasets import Dataset
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [42]:
df = pd.read_csv('data/processed/AITA-Reddit-Dataset.csv')

In [44]:
model_name = 'bert-base-uncased'

model = transformers.BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
classifier = transformers.pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [50]:
# Tokenize the data
base_dataset = Dataset.from_pandas(df)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = base_dataset.select(range(10000)).map(tokenize_function, batched=True)

In [53]:
# Create a dataset class with LABEL and TEXT columns
dataset = dataset.train_test_split(test_size=0.2)
validation_test_dataset = dataset['test'].train_test_split(test_size=0.5)
dataset['validation'] = validation_test_dataset['train']
dataset['test'] = validation_test_dataset['test']


In [63]:
def compute_metrics(pred):
    print(pred)
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return 0, 0

In [64]:
# finetune the model
training_args = transformers.TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=500,
    # label_names = ['NTA', 'YTA', 'ESH', 'NAH']
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics    
)
trainer.train()
    

  0%|          | 0/24000 [51:33<?, ?it/s]


TypeError: BertForSequenceClassification.forward() got an unexpected keyword argument 'ESH'